In [ ]:
# Fuel Penalty Model
# Cell 1
import time
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, VecMonitor
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import TensorBoardOutputFormat
from stable_baselines3.common.utils import get_linear_fn

#  wrapper to extract custom metrics (fuel, landing error) 
class LunarWrapper(gym.Wrapper):
    def reset(self, **kwargs):
        obs, info = super().reset(**kwargs)
        return obs, info

    def step(self, action):
        obs, reward, terminated, truncated, info = super().step(action)
        # approximate “fuel used”
        fuel_used = 1.0 if action == 2 else 0.0
        info["fuel_used"] = fuel_used
        # landing error = distance from pad center
        info["landing_err"] = np.linalg.norm(obs[0:2])
        #  SHAPING EDIT: penalize fuel usage 
        k_f = 0.3                    # fuel-penalty coefficient
        reward = reward - k_f * fuel_used
        return obs, reward, terminated, truncated, info

# callback to log metrics to TensorBoard
class MetricsCallback(BaseCallback):
    def _on_training_start(self):
        if not any(isinstance(fmt, TensorBoardOutputFormat)
                   for fmt in self.logger.output_formats):
            self.logger.configure(format_strings=["tensorboard"])
    def _on_step(self) -> bool:
        for info in self.locals.get("infos", []):
            if "episode" in info:
                self.logger.record("custom/fuel_used",   info["fuel_used"])
                self.logger.record("custom/landing_err", info["landing_err"])
        return True

#  build vec-env, monitor, normalize 
def make_env():
    env = gym.make("LunarLander-v3")            # use v3
    env = LunarWrapper(env)
    return env

env = DummyVecEnv([make_env])
# ← write per‑episode stats to monitor_03.csv
env = VecMonitor(env, filename="monitor_03.csv")
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.0)

# instantiate PPO with fine-tuned hyperparameters 
model = PPO(
    policy="MlpPolicy",
    env=env,
    tensorboard_log="./ppo_lunar_tb",
    verbose=1,
    n_steps=4096,
    batch_size=256,
    learning_rate=get_linear_fn(5e-4, 1e-5, 0.1),
    clip_range=0.15,
    ent_coef=0.005,
    policy_kwargs=dict(net_arch=[dict(pi=[128,128], vf=[128,128])]),
)

# train for 1 000 000 steps, logging metrics 
model.learn(
    total_timesteps=1_000_000,
    callback=MetricsCallback(),
    tb_log_name="fine_tune"
)

# save final model and normalization stats
model.save("ppo_lunar_with_metrics")
env.save("ppo_vecnormalize.pkl")
print("Training complete and model saved.")

<frozen importlib._bootstrap>:228: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.
c:\Users\a13hk\anaconda3\envs\ll_env\lib\site-packages\stable_baselines3\common\policies.py:486: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


Using cpu device
Logging to ./ppo_lunar_tb\fine_tune_4
-----------------------------------
| custom/            |            |
|    fuel_used       | 0          |
|    landing_err     | 0.11691982 |
| rollout/           |            |
|    ep_len_mean     | 91.5       |
|    ep_rew_mean     | -180       |
| time/              |            |
|    fps             | 1272       |
|    iterations      | 1          |
|    time_elapsed    | 3          |
|    total_timesteps | 4096       |
-----------------------------------
------------------------------------------
| custom/                 |              |
|    fuel_used            | 0            |
|    landing_err          | 0.15887979   |
| rollout/                |              |
|    ep_len_mean          | 92.4         |
|    ep_rew_mean          | -169         |
| time/                   |              |
|    fps                  | 1197         |
|    iterations           | 2            |
|    time_elapsed         | 6            |
|   

In this cell I set up a custom Gym wrapper that logs fuel usage and landing error, then subtracts a small penalty each time the main engine fires. I build the vectorized + normalized environment, configure PPO’s hyperparameters, and let it train for one million timesteps. At the end I save both the model and the normalization stats. The model was trained for one million steps rather than 500K like the baseline so that I could more confidently rule out that the model only needed more training to surpass a plateau.

In [ ]:
# redefine wrapper
class LunarWrapper(gym.Wrapper):
    def reset(self, **kwargs):
        obs, info = super().reset(**kwargs)
        return obs, info
    def step(self, action):
        obs, reward, terminated, truncated, info = super().step(action)
        return obs, reward, terminated, truncated, info

def play_back(model_path, n_steps=500, sleep=0.016):
    # recreate identical vec env + normalization
    def make_env():
        env = gym.make("LunarLander-v3", render_mode="human")
        return LunarWrapper(env)

    eval_env = DummyVecEnv([make_env])
    eval_env = VecMonitor(eval_env)
    # load saved stats so obs scaling matches training
    eval_env = VecNormalize.load("ppo_vecnormalize.pkl", eval_env)

    # load model
    model = PPO.load(model_path, env=eval_env)

    obs = eval_env.reset()
    for i in range(n_steps):
        action, i = model.predict(obs, deterministic=True)
        obs, rewards, dones, infos = eval_env.step(action)
        eval_env.render()
        time.sleep(sleep)
        if dones[0]:
            break
    eval_env.close()

# watch the trained agent
print("Playing back final policy:")
play_back("ppo_lunar_with_metrics.zip")



Playing back final policy:


Here I reload the trained policy and spin up the same normalization pipeline (for observations only). Then I render the LunarLander in real time so you can watch how the fuel‑penalized agent actually lands.

In [ ]:
# Cell 3: Quantitative evaluation + CSV export

import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, VecMonitor
from statistics import mean, stdev
import pandas as pd

# bring in LunarWrapper so info["fuel_used"] exists
class LunarWrapper(gym.Wrapper):
    def reset(self, **kwargs):
        obs, info = super().reset(**kwargs)
        return obs, info
    def step(self, action):
        obs, reward, term, trunc, info = super().step(action)
        info["fuel_used"] = 1.0 if action == 2 else 0.0
        return obs, reward, term, trunc, info

def make_eval_env():
    base = gym.make("LunarLander-v3")
    return LunarWrapper(base)

# build vec env & load normalization
eval_env = DummyVecEnv([make_eval_env])
eval_env = VecMonitor(eval_env)
eval_env = VecNormalize.load("ppo_vecnormalize.pkl", eval_env)
eval_env.training    = False
eval_env.norm_reward = False

# load model
model = PPO.load("ppo_lunar_with_metrics.zip", env=eval_env)

# evaluation loop
def evaluate(model, env, n_episodes=50):
    rewards, fuels, successes = [], [], 0
    for i in range(n_episodes):
        obs = env.reset()    # only obs returned in VecEnv
        done = False
        R, F = 0.0, 0.0
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, dones, infos = env.step(action)
            R += reward[0]
            F += infos[0]["fuel_used"]
            done = dones[0]
        rewards.append(R)
        fuels.append(F)
        if R >= 200:
            successes += 1
    return rewards, fuels, successes

rewards, fuels, successes = evaluate(model, eval_env, n_episodes=50)
mean_r, mean_f = mean(rewards), mean(fuels)
succ_rate = successes / 50

print(f"Mean reward:    {mean_r:.1f} ± {stdev(rewards):.1f}")
print(f"Mean fuel used: {mean_f:.1f} ± {stdev(fuels):.1f}")
print(f"Success rate:   {successes}/50 = {succ_rate:.0%}")

# write metrics CSV for bar chart
df = pd.DataFrame([{
    "experiment":   "ppo_fuel_penalty",
    "mean_reward":  mean_r,
    "mean_fuel":    mean_f,
    "success_rate": succ_rate
}])
df.to_csv("03_fuel_penalty_metrics.csv", index=False)
print("→ metrics written to 03_fuel_penalty_metrics.csv")


Mean reward:    101.3 ± 112.5
Mean fuel used: 184.7 ± 30.4
Success rate:   8/50 = 16%
→ metrics written to 03_fuel_penalty_metrics.csv


Finally, I run fifty evaluation episodes without rendering and report the average reward and total fuel used. This gives a quick quantitative check on how well the fuel‑penalized policy performs compared to the baseline.